# German Lawyer 

A Jupyter notebook to help navigate the residency law in Germany. This project uses local embeddings and models to do RAG (Retreival Augmented Generation) over the German residency law. This means that the model is living locally on the computer, the embeddings are done locally, and the querying is done locally.

You can ask questions like:

* What are the requirements for a Blue Card?
* What are the requirements for a student visa?
* What are the requirements for a work visa?

I've taken the Aufenthaltsgesetz and Aufenthaltsverordnung from Gesetze im Internet as XML and using the Unstructured XML loader, I've loaded them in as a LangChain document.

## Project Steps

1. Load the XML files into a LangChain document
2. Split the document into sections
3. Embeddings
4. Vector Store
5. LLM Setup (Prompt Template & Querying)

### 1: Use LangChain Unstructured XML Loader to Load in the German Residence Law

In [29]:
from langchain.document_loaders import UnstructuredXMLLoader, TextLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

from langchain.embeddings import OllamaEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import Chroma, Qdrant

from langchain.llms import Ollama

from langchain.chains import RetrievalQA   
from langchain.prompts import PromptTemplate 

import time

from langchain.chains import ConversationalRetrievalChain


### 2: Load & Split the Text

In [30]:
# German Residence Law
# source: https://www.gesetze-im-internet.de/aufenthv/BJNR294510004.html
file = "german-law/laws/Aufenthaltsverordnung/BJNR294510004.xml"

aufenthg = "german-law/laws/Aufenthalt-BJNR195010004.xml"

# # load German Residence Law XML file with UnstructuredXMLLoader
loader = UnstructuredXMLLoader(file_path = file)
old_docs = loader.load()

In [14]:
len(old_docs)

1

In [10]:
files = [file, aufenthg]

In [31]:
# Load multiple files into the document 
docs = []
for file in files: 
    # load German Residence Law XML file with UnstructuredXMLLoader
    loader = UnstructuredXMLLoader(file_path = file)
    docs += loader.load()

In [18]:
type(docs[1])

langchain_core.documents.base.Document

**Recursive Character Text Splitter**

Use recursive character text splitter to split texts into chunks of 1000

In [32]:
# Try with the RecursiveCharacterTextSplitter

r_text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, chunk_overlap  = 150)
r_texts = r_text_splitter.split_documents(docs)
# r_texts_old = r_text_splitter.create_documents([docs[0].page_content])


In [25]:
type(r_texts_old)

list

In [28]:
print(type(r_texts))
print(type(r_texts[0]))

<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [ ]:
# Try with the CharacterTextSplitter

c_text_splitter = CharacterTextSplitter(chunk_size = 1500, chunk_overlap  = 150)
c_texts = c_text_splitter.create_documents([docs[0].page_content])


### 3: Create Vectorstore

In [33]:
# OpenAI Embeddings, Chroma as vectorstore
openai_vectorstore = Chroma.from_documents(documents = r_texts, embedding=OpenAIEmbeddings())
retreiver = openai_vectorstore.as_retriever()

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-zgR8M***************************************SjY1. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### Local Embeddings
(Note: takes about 9 minutes / document)

In [35]:
# Ollama Embeddings (openhermes2.5), Qdrant as vectorstore 
# Note: (Chroma does not work, as Ollama creates 4096-dimensional vectors and Chroma accepts 1536-dimensional vectors only)

# loader = TextLoader("/Users/ingrid/Developer/GitHub/lawyer/README.md")
# docs = loader.load()

# test_text_splitter = CharacterTextSplitter(chunk_size = 1500, chunk_overlap  = 150)
# test_texts = test_text_splitter.create_documents([docs[0].page_content])

# REMEMBER: set the documents= to the docs that you want to embed (this function is expensive)

ollama_vectorstore = Qdrant.from_documents(
    documents=r_texts, 
    embedding=OllamaEmbeddings(
        model="llama2",
        show_progress=True,
        ),
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="texts",
)


OllamaEmbeddings:   0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
ollama_retreiver = ollama_vectorstore.as_retriever()

### 4: LLM Setup

**LLM Setup**

In [ ]:
# Temporarily set the model to 'mistral'
llm = Ollama(model='openhermes2.5-mistral:7b-q5_K_M')

**Retrieval QA Prompt**

#### Let the Not a Lawyer be a Not Lawyer

In [ ]:
# define a function which takes as inputs the llm, embeddings, and outputs the result (printed)
# ideally log as tags which llm and embeddings was used, allow me to categorize outputs as (good, not good, or comment in some ways)
import time 
def test_llm(vectorstore, model, question):

    start = time.time()

    # build prompt 
    template = """
        You are a professional, courteous, helpful AI legal assistant for question-answering tasks about residency law for people living in, or considering moving to Germany. 
        Use the following pieces of retrieved context from the German Law (delimited in $$$ $$$)to answer the question. Always cite the source of your answer.
        If you don't know the answer, just say that you don't know. Do not make anything up!
        Always cite the source of your answer! And, don't forget to empathize with the user - they are probably stressed out and need help!
        Question: {question} 

        Context: $$$ {context} $$$

        Answer:

        """

    # create prompt template
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    # set qa chain
    qa_chain_mr = RetrievalQA.from_chain_type(
        Ollama(model=model), 
        retriever = vectorstore.as_retriever(),
        chain_type="stuff", 
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

    # get the result
    result = qa_chain_mr({"query": question})

    # print the result
    print(result["result"])

    end = time.time()
    elapsed_time = end - start
    print("The function took", elapsed_time, "seconds to run.")


## Use Ollama to Install the local Models You Want to Use

Run the following commands in your terminal to install the models you want to use:

`ollama run openhermes2.5-mistral:7b-q5_K_M`

`ollama run llama2`

`ollama run mistral`

In [ ]:
frage = "How can I move to germany? I'm from the United states."
test_llm(openai_vectorstore, 'openhermes2.5-mistral:7b-q5_K_M', frage)

In [ ]:
frage = "I just got a job in Germany paying me 80,000 euros annually. What are my options for a residence permit?"
test_llm(ollama_vectorstore, 'mistral', frage)

In [ ]:
frage = "What are the requirements for a Blue Card?"
test_llm(openai_vectorstore, 'llama2', frage)

In [ ]:
frage = "What are the requirements for a Blue Card?"
test_llm(openai_vectorstore, 'openhermes2.5-mistral:7b-q5_K_M', frage)

In [ ]:
frage = "What are the requirements for a Blue Card?"
test_llm(openai_vectorstore, 'mistral', frage)

In [ ]:
frage = "What are the requirements for a Blue Card?"
test_llm(ollama_vectorstore, 'mistral', frage)

In [ ]:
frage = "What are the requirements for a Blue Card?"
test_llm(ollama_vectorstore, 'llama2', frage)

In [ ]:
frage = "What are the requirements for a Blue Card?"
test_llm(ollama_vectorstore, 'openhermes2.5-mistral:7b-q5_K_M', frage)

In [ ]:
frage = "How can a resident of Germany obtain citizenship?"
test_llm(ollama_vectorstore, 'mistral', frage)

In [ ]:
test_llm(ollama_vectorstore, 'llama2', frage)

### Findings:

Recursive Text Splitter
 * mistral: 19.5s
 * llama2: 26.2s

 Text splitter
 * mistral: 26.5s
 * llama2: 79.7s

 Conclusion: mistral is faster, recursive character text splitter is faster. Why? No idea.

### Set up memory

In [18]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

**Define a function to run the conversational retrieval chain (including memory)**

In [19]:
def test_llm_inkl_memory(vectorstore, model, question):

    retriever=vectorstore.as_retriever()
    qa = ConversationalRetrievalChain.from_llm(
        Ollama(model=model),
        retriever=retriever,
        memory=memory
    )
    result = qa({"question": question}) 
    print(result['answer'])

In [20]:
qyery = "can i travel outside the EU with a blue card valid for less than 6 months?"

test_llm_inkl_memory(openai_vectorstore, 'mistral', qyery)


To answer the question, we need to consider whether it is possible to travel outside the EU with a Blue Card that is valid for less than six months. The Blue Card is an EU residence permit that allows its holder to work and live in another member state of the EU. However, it does not grant its holder the right to leave the EU for more than six months within a twelve-month period without a valid Schengen visa or residence permit for the country they plan to visit.

Therefore, if your Blue Card is only valid for less than six months, you may not be able to travel outside the EU with it. You would need to either apply for an extension of your Blue Card or obtain a separate Schengen visa or residence permit for the country you plan to visit. It's important to note that the rules and requirements for obtaining a Schengen visa or residence permit can vary depending on the country you are planning to visit, so it's best to check with the appropriate authorities in that country before making 

In [ ]:
# Get the answer
question = "How do I get a bluecard?"
test_llm_inkl_memory(openai_vectorstore, 'mistral', question)

In [ ]:

question = "I don't already have a bluecard, but I just got a job offer for 100k. Can I get a bluecard?"
test_llm_inkl_memory(openai_vectorstore, 'mistral', question)

In [ ]:

question = "How do i get one if i haven't had one before?"
test_llm_inkl_memory(openai_vectorstore, 'mistral', question)